# Evaluation Pipeline

Here, I am implementing the metrices used for evaluating the evaluation quality

In [1]:
#imports 

import os
import sys

path_to_add = os.path.abspath(os.path.join(os.getcwd(), '..'))
if path_to_add not in sys.path:
    sys.path.append(path_to_add)


import numpy as np
import pandas as pd
import pickle

from Utilities.utilities import *
from Utilities.lime_segmentation import *
from Utilities.xai_evaluation import *


2024-02-25 11:06:02.562797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-25 11:06:02.562827: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/pknab/miniconda3/envs/envi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
date_path = "FEL_RES"

dataset_path = "../Dataset/Results/"+date_path+"/eval_results.pkl"
    
dataset = pickle.load(open(dataset_path, "rb"))

small_dataset= get_small_eval_dictionary(dataset)

In [3]:
lime_names = ["DSEG", "BayesLime", "SLIME", "GLIME", "LIME"]
images = np.unique(list((small_dataset.keys())))
for i in small_dataset.keys():
    domains = np.unique(list((small_dataset[i].keys())))
sorted_dictionary = {}

ground_truths = []
groundtruth_test = []

for local_domain in domains:
    sorted_dictionary[local_domain] = {}
    for local_image in images:
        for in_keys in list(small_dataset[local_image]["groundtruth"].keys()):
            if "ground" in in_keys.lower():
                ground_truths.append(small_dataset[local_image]["groundtruth"][in_keys][1])
            break
        for in_keys in list(small_dataset[local_image]["contrastivity"].keys()):
            if "groundtruth_test" in in_keys.lower():
                groundtruth_test.append(small_dataset[local_image]["contrastivity"][in_keys][1])
            break
        for local_name in small_dataset[local_image][local_domain].keys():
            #print(small_dataset[local_image])
            if local_image not in sorted_dictionary[local_domain].keys():
                sorted_dictionary[local_domain][local_image] = {}
            for local_metric in lime_names:
                if local_metric.lower() not in sorted_dictionary[local_domain][local_image].keys():
                    sorted_dictionary[local_domain][local_image][local_metric.lower()] = {}
            for local_metric in lime_names:
                if local_metric.lower() in local_name.lower():
                    sorted_dictionary[local_domain][local_image][local_metric.lower()][local_name] = small_dataset[local_image][local_domain][local_name]
                    
                    break


final_dict = {}
order_images = []
for i in list(sorted_dictionary.keys()):
    for domain in list(sorted_dictionary[i].keys()):
        domain_dict = {}
        local_dictionary = sorted_dictionary[i]
        for j in local_dictionary.keys():
            local_dict = local_dictionary[j]
            for k in local_dict.keys():
                if k not in domain_dict.keys():
                    domain_dict[k] = {}
                local_dict_ = local_dict[k]
                for l in local_dict_.keys():
                    if l not in domain_dict[k].keys():
                        domain_dict[k][l] = []
                    domain_dict[k][l].append(local_dict_[l])
                    
        final_dict[i] = domain_dict
    
eval_dict = {}
for i in final_dict.keys():
    local_df = {}
    for j in final_dict[i].keys():
        for k in final_dict[i][j].keys():
            if len(np.array(final_dict[i][j][k]).shape) == 2:
                if "preservation_noise" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1
                elif "deletion_noise" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                
                elif "preservation_check_contrast" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == groundtruth_test[num]:
                            local_df[k] += 1
                elif "deletion_check_contrast" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != groundtruth_test[num]:
                            local_df[k] += 1
                
                elif "preservation" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1

                elif "single_deletion" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1
                elif "deletion" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                elif "model_random" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                elif "eval_random" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
            else:
                try:
                    local_df[k] = np.mean(final_dict[i][j][k])
                except:
                    pass
                
    eval_dict[i] = local_df

def dataframe_results(data):
    df = pd.DataFrame(data.items(), columns=['Metric', 'Value'])

    # Splitting the 'Metric' column to extract additional categorical information
    df[['Method', 'Metric_Type']] = df['Metric'].str.split('_', n=1, expand=True)

    # Drop the original 'Metric' column as its information has been extracted
    df.drop('Metric', axis=1, inplace=True)

    # Reordering the columns for better readability
    df = df[['Method', 'Metric_Type', 'Value']]

    df_pivot = df.pivot(index='Metric_Type', columns='Method', values='Value')

    # Display the pivoted DataFrame
    return df_pivot
    
eval_dict.keys()

dict_keys(['consistency', 'contrastivity', 'correctness', 'groundtruth', 'output_completness'])

## Overview

In [4]:
dataframe_results(eval_dict["groundtruth"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
segments,16.70000,15.85000,16.70000,16.700000,16.7000
time,18.70425,45.96041,19.40317,19.382695,17.5385


In [5]:
df_color = ColoredDF()
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 0)


,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n06794110' 'street_sign' '0.14786814'],39.038900,18.000000,26.134400,17.000000,20.147900,17.000000,17.116800,17.000000,13.317300,17.000000
000000000785.jpg,['n04228054' 'ski' '0.9835006'],46.161300,5.000000,20.054200,7.000000,18.787800,7.000000,15.794100,7.000000,15.367600,7.000000
000000289059.jpg,['n04039381' 'racket' '0.9071478'],57.038100,7.000000,21.151500,8.000000,16.135200,8.000000,15.856000,8.000000,21.698800,8.000000
n02134084_ice_bear.JPEG,['n02134084' 'ice_bear' '0.52830356'],40.224300,11.000000,23.251800,10.000000,17.983100,10.000000,18.621900,10.000000,18.129300,10.000000
n03891251_park_bench.JPEG,['n03891251' 'park_bench' '0.59677887'],42.390400,14.000000,16.647000,16.000000,17.822700,16.000000,17.995000,16.000000,17.810900,16.000000
n02480855_gorilla.JPEG,['n02480855' 'gorilla' '0.2135582'],58.068300,31.000000,23.548800,35.000000,21.982800,35.000000,19.828100,35.000000,42.351200,35.000000
n02988304_CD_player.JPEG,['n02988304' 'CD_player' '0.33121333'],43.163500,7.000000,15.542900,9.000000,15.532100,9.000000,15.370300,9.000000,13.999700,9.000000
000000290619.jpg,['n03133878' 'Crock_Pot' '0.104617536'],50.738600,5.000000,16.088700,8.000000,38.818600,8.000000,32.270100,8.000000,29.911700,8.000000
n04008634_projectile.JPEG,['n04008634' 'projectile' '0.5816481'],46.856000,36.000000,17.930700,36.000000,16.677100,36.000000,17.379200,36.000000,17.528000,36.000000
n02981792_catamaran.JPEG,['n02981792' 'catamaran' '0.64228153'],38.184500,10.000000,24.601700,14.000000,24.670900,14.000000,17.811700,14.000000,16.349000,14.000000


## Correctness

In [6]:
dataframe_results(eval_dict["correctness"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
Compactness,0.15485,0.200115,0.15230,0.16037,0.175735
incremental_deletion,0.43579,0.273155,0.36945,0.36873,0.422455
prediction_eval_random,20.00000,19.000000,18.00000,20.00000,19.000000
prediction_model_random,17.00000,18.000000,20.00000,18.00000,19.000000
prediction_single_deletion,4.00000,4.000000,3.00000,4.00000,3.000000


In [7]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 1)

,DSEG_prediction_model_random,LIME_prediction_model_random,BayesLIME_prediction_model_random,SLIME_prediction_model_random,GLIME_prediction_model_random,DSEG_prediction_eval_random,LIME_prediction_eval_random,BayesLIME_prediction_eval_random,SLIME_prediction_eval_random,GLIME_prediction_eval_random,DSEG_incremental_deletion,LIME_incremental_deletion,BayesLIME_incremental_deletion,SLIME_incremental_deletion,GLIME_incremental_deletion,DSEG_prediction_single_deletion,DSEG_Compactness,LIME_prediction_single_deletion,LIME_Compactness,BayesLIME_prediction_single_deletion,BayesLIME_Compactness,SLIME_prediction_single_deletion,SLIME_Compactness,GLIME_prediction_single_deletion,GLIME_Compactness,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n03970156' 'plunger' '0.02175259'],['n03126707' 'crane' '0.038725555'],['n03467068' 'guillotine' '0.034578353'],['n06359193' 'web_site' '0.031703807'],['n02825657' 'bell_cote' '0.07690366'],['n03532672' 'hook' '0.025847964'],['n03796401' 'moving_van' '0.019853413'],['n04258138' 'solar_dish' '0.032897495'],['n02825657' 'bell_cote' '0.13475819'],['n06794110' 'street_sign' '0.06666461'],0.034400,0.000000,0.000000,0.000000,0.000000,['n09332890' 'lakeside' '0.14000916'],0.165900,['n03976657' 'pole' '0.2232769'],0.064100,['n09421951' 'sandbar' '0.09547448'],0.192600,['n09421951' 'sandbar' '0.09547448'],0.192600,['n09421951' 'sandbar' '0.09547448'],0.192600,['n06794110' 'street_sign' '0.14786814'],39.038900,18.000000,26.134400,17.000000,20.147900,17.000000,17.116800,17.000000,13.317300,17.000000
000000000785.jpg,['n03291819' 'envelope' '0.58706206'],['n03291819' 'envelope' '0.114684515'],['n03291819' 'envelope' '0.09740405'],['n03291819' 'envelope' '0.41522413'],['n04033901' 'quill' '0.092756934'],['n03291819' 'envelope' '0.13630709'],['n03291819' 'envelope' '0.114684515'],['n03291819' 'envelope' '0.09740405'],['n03291819' 'envelope' '0.09740405'],['n04228054' 'ski' '0.0461742'],0.130200,0.646900,0.646900,0.646900,0.646900,['n04228054' 'ski' '0.49055734'],0.630300,['n04273569' 'speedboat' '0.107799776'],0.263800,['n04273569' 'speedboat' '0.107799776'],0.263800,['n04273569' 'speedboat' '0.107799776'],0.263800,['n04273569' 'speedboat' '0.107799776'],0.263800,['n04228054' 'ski' '0.9835006'],46.161300,5.000000,20.054200,7.000000,18.787800,7.000000,15.794100,7.000000,15.367600,7.000000
000000289059.jpg,['n04039381' 'racket' '0.051801126'],['n03595614' 'jersey' '0.021332335'],['n03291819' 'envelope' '0.10280368'],['n03291819' 'envelope' '0.040519882'],['n02916936' 'bulletproof_vest' '0.039926965'],['n04039381' 'racket' '0.5830325'],['n03291819' 'envelope' '0.040519882'],['n03595614' 'jersey' '0.021332335'],['n03291819' 'envelope' '0.040519882'],['n03291819' 'envelope' '0.12783481'],0.365300,0.122000,0.122000,0.122000,0.122000,['n04039381' 'racket' '0.12964143'],0.057300,['n04039381' 'racket' '0.29108208'],0.160600,['n04039381' 'racket' '0.29108208'],0.160600,['n04039381' 'racket' '0.29108208'],0.160600,['n04039381' 'racket' '0.29108208'],0.160600,['n04039381' 'racket' '0.9071478'],57.038100,7.000000,21.151500,8.000000,16.135200,8.000000,15.856000,8.000000,21.698800,8.000000
n02134084_ice_bear.JPEG,['n03109150' 'corkscrew' '0.04381132'],['n03498962' 'hatchet' '0.030425396'],['n02134084' 'ice_bear' '0.31446496'],['n03291819' 'envelope' '0.03914801'],['n03291819' 'envelope' '0.03914801'],['n04523525' 'vault' '0.05274052'],['n03291819' 'envelope' '0.05009985'],['n03291819' 'envelope' '0.068028234'],['n03498962' 'hatchet' '0.030425396'],['n04522168' 'vase' '0.026625821'],0.437100,0.440500,0.440500,0.440500,0.440500,['n02894605' 'breakwater' '0.060918905'],0.175200,['n02894605' 'breakwater' '0.25962472'],0.281300,['n02894605' 'breakwater' '0.25962472'],0.281300,['n02894605' 'breakwater' '0.25962472'],0.281300,['n02894605' 'breakwater' '0.25962472'],0.281300,['n02134084' 'ice_bear' '0.52830356'],40.224

## Contrastivity


In [8]:
dataframe_results(eval_dict["contrastivity"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_check_contrast,4,9,5,6,5
deletion_noise_check,8,13,9,7,7
preservation_check_contrast,6,7,6,8,6
preservation_noise_check,5,7,5,5,4
single_deletion,3,6,3,3,4


In [9]:

df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 2)

,Groundtruth_test,DSEG_preservation_check_contrast,DSEG_deletion_check_contrast,LIME_preservation_check_contrast,LIME_deletion_check_contrast,BayesLIME_preservation_check_contrast,BayesLIME_deletion_check_contrast,SLIME_preservation_check_contrast,SLIME_deletion_check_contrast,GLIME_preservation_check_contrast,GLIME_deletion_check_contrast,DSEG_single_deletion,LIME_single_deletion,BayesLIME_single_deletion,SLIME_single_deletion,GLIME_single_deletion,DSEG_preservation_noise_check,DSEG_deletion_noise_check,LIME_preservation_noise_check,LIME_deletion_noise_check,BayesLIME_preservation_noise_check,BayesLIME_deletion_noise_check,SLIME_preservation_noise_check,SLIME_deletion_noise_check,GLIME_preservation_noise_check,GLIME_deletion_noise_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n06794110' 'street_sign' '0.1489823'],['n06874185' 'traffic_light' '0.26531032'],['n02930766' 'cab' '0.3758966'],['n03891332' 'parking_meter' '0.16429572'],['n02747177' 'ashcan' '0.10428119'],['n02930766' 'cab' '0.5357982'],['n06794110' 'street_sign' '0.15460926'],['n03141823' 'crutch' '0.2118608'],['n06794110' 'street_sign' '0.12320476'],['n09421951' 'sandbar' '0.08483186'],['n03976657' 'pole' '0.11732076'],['n09332890' 'lakeside' '0.34543875'],['n03976657' 'pole' '0.5926826'],['n09421951' 'sandbar' '0.64605445'],['n09421951' 'sandbar' '0.64605445'],['n09421951' 'sandbar' '0.64605445'],['n06794110' 'street_sign' '0.24993606'],['n03891332' 'parking_meter' '0.16983911'],['n02930766' 'cab' '0.15210317'],['n03976657' 'pole' '0.16443662'],['n02930766' 'cab' '0.15210317'],['n03976657' 'pole' '0.16443662'],['n02930766' 'cab' '0.15210317'],['n03976657' 'pole' '0.16443662'],['n02930766' 'cab' '0.20041406'],['n03976657' 'pole' '0.14316112'],['n06794110' 'street_sign' '0.14786814'],39.038900,18.000000,26.134400,17.000000,20.147900,17.000000,17.116800,17.000000,13.317300,17.000000
000000000785.jpg,['n04228054' 'ski' '0.8964271'],['n04228054' 'ski' '0.91820824'],['n04228054' 'ski' '0.79289573'],['n04228054' 'ski' '0.92801976'],['n04228054' 'ski' '0.9014095'],['n04228054' 'ski' '0.92801976'],['n04228054' 'ski' '0.9014095'],['n04228054' 'ski' '0.92801976'],['n04228054' 'ski' '0.9014095'],['n04228054' 'ski' '0.92801976'],['n04228054' 'ski' '0.9014095'],['n04228054' 'ski' '0.7284479'],['n02981792' 'catamaran' '0.16849601'],['n02981792' 'catamaran' '0.16849601'],['n02981792' 'catamaran' '0.16849601'],['n02981792' 'catamaran' '0.16849601'],['n04228054' 'ski' '0.88030833'],['n04208210' 'shovel' '0.30303508'],['n04228054' 'ski' '0.15479687'],['n04228054' 'ski' '0.93630517'],['n04228054' 'ski' '0.15479687'],['n04228054' 'ski' '0.93630517'],['n04228054' 'ski' '0.15479687'],['n04228054' 'ski' '0.93630517'],['n04228054' 'ski' '0.15479687'],['n04228054' 'ski' '0.93630517'],['n04228054' 'ski' '0.9835006'],46.161300,5.000000,20.054200,7.000000,18.787800,7.000000,15.794100,7.000000,15.367600,7.000000
000000289059.jpg,['n04039381' 'racket' '0.8029209'],['n04039381' 'racket' '0.8551795'],['n04039381' 'racket' '0.58829564'],['n04039381' 'racket' '0.8433766'],['n04039381' 'racket' '0.81327224'],['n04039381' 'racket' '0.8433766'],['n04039381' 'racket' '0.81327224'],['n04039381' 'racket' '0.8433766'],['n04039381' 'racket' '0.81327224'],['n04039381' 'racket' '0.8433766'],['n04039381' 'racket' '0.81327224'],['n04039381' 'racket' '0.47391814'],['n04039381' 'racket' '0.70222837'],['n04039381' 'racket' '0.70222837'],['n04039381' 'racket' '0.70222837'],['n04039381' 'racket' '0.70222837'],['n04039381' 'racket' '0.8119202'],['n02790996' 'barbell' '0.1955432'],['n04039381' 'racket' '0.8508537'],['n04039381' 'racket' '0.7053472'],['n04039381' 'racket' '0.8508537'],['n04039381' 'racket' '0.7053472'],['n04039381' 'racket' '0.8508537'],['n04039381' 'racket' '0.7053472'],['n04039381' 'racket' '0.8508537'],['n04039381' 'racket' '0.7053472'],['n04039381' 'racket' '0.9071478'],57.0

## Output Completenes


In [10]:
dataframe_results(eval_dict["output_completness"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_check,7,11,6,7,6
preservation_check,6,6,4,7,5


In [11]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 3)

,DSEG_preservation_check,DSEG_deletion_check,LIME_preservation_check,LIME_deletion_check,BayesLIME_preservation_check,BayesLIME_deletion_check,SLIME_preservation_check,SLIME_deletion_check,GLIME_preservation_check,GLIME_deletion_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n03355925' 'flagpole' '0.09505846'],['n04486054' 'triumphal_arch' '0.121381834'],['n03976657' 'pole' '0.03596421'],['n03877845' 'palace' '0.046919275'],['n03355925' 'flagpole' '0.02410998'],['n06794110' 'street_sign' '0.12277372'],['n04008634' 'projectile' '0.026335236'],['n06794110' 'street_sign' '0.0705432'],['n09428293' 'seashore' '0.01912986'],['n06794110' 'street_sign' '0.070707634'],['n06794110' 'street_sign' '0.14786814'],39.038900,18.000000,26.134400,17.000000,20.147900,17.000000,17.116800,17.000000,13.317300,17.000000
000000000785.jpg,['n04228054' 'ski' '0.87204546'],['n04228054' 'ski' '0.5836076'],['n04228054' 'ski' '0.076070465'],['n04228054' 'ski' '0.93774563'],['n04228054' 'ski' '0.076070465'],['n04228054' 'ski' '0.93774563'],['n04228054' 'ski' '0.076070465'],['n04228054' 'ski' '0.93774563'],['n04228054' 'ski' '0.076070465'],['n04228054' 'ski' '0.93774563'],['n04228054' 'ski' '0.9835006'],46.161300,5.000000,20.054200,7.000000,18.787800,7.000000,15.794100,7.000000,15.367600,7.000000
000000289059.jpg,['n04039381' 'racket' '0.28504896'],['n04039381' 'racket' '0.65856916'],['n04039381' 'racket' '0.722895'],['n04039381' 'racket' '0.4810981'],['n04039381' 'racket' '0.722895'],['n04039381' 'racket' '0.4810981'],['n04039381' 'racket' '0.722895'],['n04039381' 'racket' '0.4810981'],['n04039381' 'racket' '0.722895'],['n04039381' 'racket' '0.4810981'],['n04039381' 'racket' '0.9071478'],57.038100,7.000000,21.151500,8.000000,16.135200,8.000000,15.856000,8.000000,21.698800,8.000000
n02134084_ice_bear.JPEG,['n03467068' 'guillotine' '0.027893737'],['n02134084' 'ice_bear' '0.13813636'],['n02231487' 'walking_stick' '0.07650201'],['n02134084' 'ice_bear' '0.68051416'],['n02231487' 'walking_stick' '0.07650201'],['n02134084' 'ice_bear' '0.68051416'],['n02231487' 'walking_stick' '0.07650201'],['n02134084' 'ice_bear' '0.68051416'],['n02231487' 'walking_stick' '0.07650201'],['n02134084' 'ice_bear' '0.68051416'],['n02134084' 'ice_bear' '0.52830356'],40.224300,11.000000,23.251800,10.000000,17.983100,10.000000,18.621900,10.000000,18.129300,10.000000
n03891251_park_bench.JPEG,['n03794056' 'mousetrap' '0.3071331'],['n09332890' 'lakeside' '0.2480812'],['n01930112' 'nematode' '0.02983381'],['n03891251' 'park_bench' '0.47851837'],['n03794056' 'mousetrap' '0.08657558'],['n09332890' 'lakeside' '0.16329622'],['n01930112' 'nematode' '0.02983381'],['n03891251' 'park_bench' '0.47851837'],['n03692522' 'loupe' '0.012331071'],['n03891251' 'park_bench' '0.43855402'],['n03891251' 'park_bench' '0.59677887'],42.390400,14.000000,16.647000,16.000000,17.822700,16.000000,17.995000,16.000000,17.810900,16.000000
n02480855_gorilla.JPEG,['n02058221' 'albatross' '0.044289064'],['n01616318' 'vulture' '0.06908879'],['n02480855' 'gorilla' '0.11434337'],['n03000684' 'chain_saw' '0.028094862'],['n03804744' 'nail' '0.03572433'],['n02480855' 'gorilla' '0.18833655'],['n03804744' 'nail' '0.03572433'],['n02480855' 'gorilla' '0.18833655'],['n03804744' 'nail' '0.02330488'],['n02480855' 'gorilla' '0.19873331'],['n02480855' 'gorilla' '0.2135582'],58.068300,31.000000,23.548800,35.000000,21.982800,35.000000,19.828100,35.000000,42.351200,35.000000
n02988304_CD_player.JPEG,['n02988304' 'CD_player' '0.16739745'],['n02910353' 'buckle' '0.10591592'],['n04127249' 'safety_pin' '0.075679086'],['n02988304' 'CD_player' '0.25239587'],['n04127249' 'safety_pin' '0.075679086'],['n02988304' 'CD_player' '0.25239587'],['n04141975' 'scale' '0.14489721'],['n02910353' 'buckle' '0.07523088'],['n04141975' 'scale' '0.14489721'],['n02910353' 'buckle' '0.07523088'],['n02988304' 'CD_player' '0.33121333'],43.163500,7.0

## Consistency

In [12]:
dataframe_results(eval_dict["consistency"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_noise_check,9.00000,12.000000,8.00000,9.000000,8.000000
preservation_noise_check,2.00000,10.000000,2.00000,3.000000,2.000000
stability,0.02163,0.022055,0.01727,0.021575,0.022265


In [13]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 4)

,DSEG_stability,LIME_stability,BayesLIME_stability,SLIME_stability,GLIME_stability,DSEG_preservation_noise_check,DSEG_deletion_noise_check,LIME_preservation_noise_check,LIME_deletion_noise_check,BayesLIME_preservation_noise_check,BayesLIME_deletion_noise_check,SLIME_preservation_noise_check,SLIME_deletion_noise_check,GLIME_preservation_noise_check,GLIME_deletion_noise_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,0.008900,0.012900,0.013200,0.011100,0.014000,['n06794110' 'street_sign' '0.06356295'],['n03837869' 'obelisk' '0.13472612'],['n02825657' 'bell_cote' '0.064306065'],['n06874185' 'traffic_light' '0.071419746'],['n02825657' 'bell_cote' '0.064306065'],['n06874185' 'traffic_light' '0.071419746'],['n02825657' 'bell_cote' '0.064306065'],['n06874185' 'traffic_light' '0.071419746'],['n02825657' 'bell_cote' '0.045438144'],['n06874185' 'traffic_light' '0.050327104'],['n06794110' 'street_sign' '0.14786814'],39.038900,18.000000,26.134400,17.000000,20.147900,17.000000,17.116800,17.000000,13.317300,17.000000
000000000785.jpg,0.032700,0.030900,0.031900,0.037000,0.012100,['n04228054' 'ski' '0.9387276'],['n04228054' 'ski' '0.11199658'],['n02825657' 'bell_cote' '0.033040505'],['n04228054' 'ski' '0.9541489'],['n02825657' 'bell_cote' '0.033040505'],['n04228054' 'ski' '0.9541489'],['n02825657' 'bell_cote' '0.033040505'],['n04228054' 'ski' '0.9541489'],['n02825657' 'bell_cote' '0.033040505'],['n04228054' 'ski' '0.9541489'],['n04228054' 'ski' '0.9835006'],46.161300,5.000000,20.054200,7.000000,18.787800,7.000000,15.794100,7.000000,15.367600,7.000000
000000289059.jpg,0.032300,0.033200,0.036500,0.029500,0.019400,['n04039381' 'racket' '0.77632725'],['n04296562' 'stage' '0.08965765'],['n04039381' 'racket' '0.76560163'],['n04039381' 'racket' '0.5202345'],['n04039381' 'racket' '0.76560163'],['n04039381' 'racket' '0.5202345'],['n04039381' 'racket' '0.76560163'],['n04039381' 'racket' '0.5202345'],['n04039381' 'racket' '0.76560163'],['n04039381' 'racket' '0.5202345'],['n04039381' 'racket' '0.9071478'],57.038100,7.000000,21.151500,8.000000,16.135200,8.000000,15.856000,8.000000,21.698800,8.000000
n02134084_ice_bear.JPEG,0.029200,0.018700,0.018300,0.020800,0.013300,['n02134084' 'ice_bear' '0.90007246'],['n02231487' 'walking_stick' '0.04935327'],['n02134084' 'ice_bear' '0.0904808'],['n02134084' 'ice_bear' '0.35653234'],['n02231487' 'walking_stick' '0.11968118'],['n02134084' 'ice_bear' '0.67728096'],['n02231487' 'walking_stick' '0.11968118'],['n02134084' 'ice_bear' '0.67728096'],['n04486054' 'triumphal_arch' '0.03340891'],['n02134084' 'ice_bear' '0.5679972'],['n02134084' 'ice_bear' '0.52830356'],40.224300,11.000000,23.251800,10.000000,17.983100,10.000000,18.621900,10.000000,18.129300,10.000000
n03891251_park_bench.JPEG,0.019200,0.025000,0.020900,0.023200,0.020800,['n04326547' 'stone_wall' '0.09386233'],['n03743016' 'megalith' '0.2740876'],['n02417914' 'ibex' '0.019973662'],['n03891251' 'park_bench' '0.25148523'],['n03794056' 'mousetrap' '0.10935997'],['n09332890' 'lakeside' '0.16611147'],['n02417914' 'ibex' '0.019973662'],['n03891251' 'park_bench' '0.25148523'],['n03743016' 'megalith' '0.03529857'],['n03891251' 'park_bench' '0.54770786'],['n03891251' 'park_bench' '0.59677887'],42.390400,14.000000,16.647000,16.000000,17.822700,16.000000,17.995000,16.000000,17.810900,16.000000
n02480855_gorilla.JPEG,0.016500,0.013400,0.014900,0.013800,0.013800,['n02441942' 'weasel' '0.25986728'],['n01843383' 'toucan' '0.049661182'],['n02481823' 'chimpanzee' '0.08688556'],['n01843383' 'toucan' '0.042231508'],['n02481823' 'chimpanzee' '0.08688556'],['n01843383' 'toucan' '0.042231508'],['n02481823' 'chimpanzee' '0.08688556'],['n01843383' 'toucan' '0.042231508'],['n02481823' 'chimpanzee' '0.08688556'],['n01843383' 'toucan' '0.042231508'],['n02480855' 'gorilla' '0.2135582'],58.068300,31.000000,23.548800,35.000000,21.982800,35.000000,19.828100,35.000000,42.3512